In [1]:
import sys
import os
import tkinter as tk
from tkinter import messagebox
import pygame
import numpy as np
from env_field import FieldEnv
import trainer
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.3, Python 3.10.14)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# 強化学習情報表示の関数定義
def show_info(t, act, rwd, done, obs, isFirst=False):
    """ 強化学習情報の表示 """
    if rwd is None:
        if isFirst:
            tt = t
        else:
            tt = t + 1
        print('')
        print(f'x({tt:d})={str(obs):s}')
    else:
        msg = (
            f'a({t:d})={act:d}, ' +
            f'r({t:d})={rwd: .2f}, ' +
            f'done({t:d})={done:}, ' +
            f'x({t + 1:d})={str(obs):s}'
        )
        print(msg)

def show_graph(pathname, target_reward=None, target_step=None):
    """
    学習曲線の表示

    Parameters
    ----------
    target_reward: float or None
        rewardの目標値に線を引く
    target_step: float or None
        stepの目標値に線を引く
    """
    hist = np.load(pathname + '.npz')
    eval_rwd = hist['eval_rwds'].tolist()
    eval_step = hist['eval_steps'].tolist()
    eval_x = hist['eval_x'].tolist()

    plt.figure(figsize=(8, 4))
    plt.subplots_adjust(hspace=0.6)

    # reward / episode
    plt.subplot(211)
    plt.plot(eval_x, eval_rwd, 'b.-')
    if target_reward is not None:
        plt.plot(
            [eval_x[0], eval_x[-1]],
            [target_reward, target_reward],
            'r:')

    plt.title('rewards / episode')
    plt.grid(axis='both')

    # steps / episode
    plt.subplot(212)
    plt.plot(eval_x, eval_step, 'b.-')
    if target_step is not None:
        plt.plot(
            [eval_x[0], eval_x[-1]],
            [target_step, target_step],
            'r:')
    plt.title('steps / episode')
    plt.xlabel('steps')
    plt.grid(axis='both')

    plt.show()


In [3]:
def Youplay(task_type):
    # 環境の準備
    env = FieldEnv()

    env.set_task_type(task_type)
    msg = '---- 操作方法 -------------------------------------\n' + \
          '[e] 前に進む [s] 左に90度回る [f] 右に90度回る\n' + \
          '[q] 終了\n' + \
          '全てのクリスタルを回収するとクリア、次のエピソードが開始\n' + \
          '---------------------------------------------------'
    print(msg)

    # 強化学習情報の初期化
    t = 0
    obs = env.reset()
    act = None
    rwd = None
    done = False

    # 開始の表示
    print('あなたのプレイ開始')
    
    env.pygame_init()

    # 強化学習情報表示
    show_info(t, act, rwd, done, obs, isFirst=True)
    
    running = True
    keyboard = None
    clock = pygame.time.Clock()

    # シミュレーション
    while running:
        # 画面表示
        env.pygame_render('[e] move [s] turn left [f] turn left ' + \
          '[q] finish\n' + \
          'When all crystals are collected, the stage is cleared,\n and the next episode begins.')

        # キーの受付と終了処理
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    running = False  # ゲームをリセット
                elif event.key == pygame.K_e:
                    act = 0 # 進む
                    keyboard = 0
                elif event.key == pygame.K_s:
                    act = 1 # 右回転
                    keyboard = 0
                elif event.key == pygame.K_f:
                    act = 2 # 右回転
                    keyboard = 0
        # キーが押された場合のみ環境を更新
        if keyboard != None:
            # 環境の更新
            rwd, done, obs = env.step(act)
            
            # 強化学習情報表示
            show_info(t, act, rwd, done, obs)
            keyboard = None  # keyをリセット
            t += 1

    pygame.quit()

In [4]:
def Agtplay(agt_type, task_type, process_type):
    # 保存用フォルダの確認・作成
    if not os.path.exists("agt_data"):
        os.mkdir("agt_data")
    
    # process_typeでパラメータをセット
    if process_type == 'learn':
        IS_LOAD_DATA = False    # 学習したデータを読み込むかどうか
        IS_LEARN = True         # 学習を行うかどうか
        IS_SHOW_GRAPH = True    # 学習曲線のグラフを表示するかどうか
        IS_SHOW_ANIME = False   # 画像によるシミュレーションの表示をするかどうか
    elif process_type == 'more':
        IS_LOAD_DATA = True
        IS_LEARN = True
        IS_SHOW_GRAPH = True
        IS_SHOW_ANIME = False
    elif process_type == 'graph':
        IS_LOAD_DATA = False
        IS_LEARN = False
        IS_SHOW_GRAPH = True
        IS_SHOW_ANIME = False
    elif process_type == 'anime':
        IS_LOAD_DATA = True
        IS_LEARN = False
        IS_SHOW_GRAPH = False
        IS_SHOW_ANIME = True
        
    else:
        print('process type が間違っています。')
        sys.exit()

    # 環境の設定 /////////////////////////////// 
    # Envインスタンス生成
    # 学習用環境
    env = FieldEnv()
    env.set_task_type(task_type)
    obs = env.reset()  # obsのサイズがエージェントのパラメータ設定で使われる

    # 評価用環境
    eval_env = FieldEnv()
    eval_env.set_task_type(task_type)

    # エージェントのパラメータ設定 /////////////// 
    # Agt 共通パラメータ
    agt_prm = {
        'n_act': env.n_act,
        'filepath': ("agt_data" + '/' +
                     "field_task" + '_' +
                     agt_type + '_' +
                     task_type)
    }
    # agt_type 別のパラメータ
    if agt_type == 'tableQ':
        agt_prm['init_val_Q'] = 0            # Q値の初期値
        agt_prm['alpha'] = 0.1               # 学習率

    elif agt_type == 'netQ':
        agt_prm['input_size'] = obs.shape    # 入力のサイズ
        agt_prm['n_dense'] = 32              # 中間層のユニット数

    elif agt_type == 'replayQ':
        agt_prm['input_size'] = obs.shape    # 入力のサイズ
        agt_prm['n_dense'] = 32              # 中間層のユニット数
        agt_prm['replay_memory_size'] = 100  # 記憶サイズ
        agt_prm['replay_batch_size'] = 20    # バッチサイズ

    elif agt_type == 'targetQ':
        agt_prm['input_size'] = obs.shape    # 入力のサイズ
        agt_prm['n_dense'] = 32              # 中間層のユニット数
        agt_prm['replay_memory_size'] = 100  # 記憶サイズ
        agt_prm['replay_batch_size'] = 20    # バッチサイズ
        agt_prm['target_interval'] = 10      # ターゲットインターバル


    # トレーナーのパラメータ設定 ////////////////
    # シミュレーション共通パラメータ
    sim_prm = {
        'n_step': 1000,         # ステップ数
        'n_episode': None,      # エピソードは終了条件にしない
        'is_learn': True,       # 学習する
        'is_eval': True,        # 評価する
        'eval_interval': 100,   # 評価のインターバル
        'eval_n_episode': 10,   # 評価のエピソード数
        'eval_n_step': None,    # 評価でステップ数は終了条件にしない
        'eval_epsilon': 0.0,    # 評価時の乱雑度
        'eval_seed': 1,         # 評価時の乱数のシード値
        'is_animation': False,  # アニメーションの表示はしない
    }

    # アニメーション共通パラメータ
    sim_anime_prm = {
        'n_step': None,         # ステップ数は終了条件にしない
        'n_episode': 100,       # エピソード数
        'seed': 1,              # 乱数のシード値を指定
        'is_eval': False,       # 評価しない
        'is_learn': False,      # 学習しない
        'is_animation': True,   # アニメーションを表示する
        'anime_delay': 0.2,     # フレーム時間(秒)
    }
    ANIME_EPSILON = 0.0

    # task_type 別のパラメータ ///////////////// 
    graph_prm = {}
    if task_type == "no_wall":
        sim_prm['n_step'] = 5000            # ステップ数
        sim_prm['eval_interval'] = 200      # 評価を何ステップごとにするか
        sim_prm['eval_n_episode'] = 100     # 評価のエピソード数
        agt_prm['epsilon'] = 0.2            # 乱雑度
        agt_prm['gamma'] = 0.9              # 割引率
        graph_prm['target_reward'] = 0.71   # グラフの赤い点線の値
        graph_prm['target_step'] = 3.87     # グラフの赤い点線の値

    elif task_type == "fixed_wall":
        sim_prm['n_step'] = 5000
        sim_prm['eval_interval'] = 200
        sim_prm['eval_n_episode'] = 1
        agt_prm['epsilon'] = 0.4
        agt_prm['gamma'] = 0.9
        graph_prm['target_reward'] = 1.0
        graph_prm['target_step'] = 12.0

    elif task_type == "random_wall":
        sim_prm['n_step'] = 50000
        sim_prm['eval_interval'] = 1000
        sim_prm['eval_n_episode'] = 100
        agt_prm['epsilon'] = 0.4
        agt_prm['gamma'] = 0.9
        graph_prm['target_reward'] = 1.6
        graph_prm['target_step'] = 22.0
        # 数値を設定すると
        # rewards/episodeがこの値を超えた時に
        # 学習シミュレーションが終了する
        sim_prm['eary_stop'] = 1.6

    # メイン ///////////////////////////////// 
    if (IS_LOAD_DATA is True) or \
            (IS_LEARN is True) or \
            (sim_prm['is_animation'] is True):

        # エージェントをインポートしてインスタンス作成
        if agt_type == 'tableQ':
            from agt_tableQ import TableQAgt as Agt
        elif agt_type == 'netQ':
            from agt_netQ import NetQAgt as Agt
        elif agt_type == 'replayQ':
            from agt_replayQ import ReplayQAgt as Agt
        elif agt_type == 'targetQ':
            from agt_targetQ import TargetQAgt as Agt
        else:
            raise ValueError('agt_type が間違っています')

        # エージェントのインスタンス生成
        agt = Agt(**agt_prm)

        # trainer インスタンス作成
        trn = trainer.Trainer(agt, env, eval_env)

        if IS_LOAD_DATA is True:
            # エージェントのデータロード
            try:
                agt.load_weights()
                trn.load_history(agt.filepath)
            except Exception as e:
                print(e)
                print('エージェントのパラメータがロードできません')
                return
                

        if IS_LEARN is True:
            # 学習
            trn.simulate(**sim_prm)
            agt.save_weights()
            trn.save_history(agt.filepath)

        if IS_SHOW_ANIME is True:
            # アニメーション
            agt.epsilon = ANIME_EPSILON
            trn.simulate(**sim_anime_prm)

    if IS_SHOW_GRAPH is True:
        # グラフ表示
        show_graph(agt_prm['filepath'], **graph_prm)

In [ ]:
def on_select():
    player_choice = player_var.get()
    task_choice = task_var.get()
    process_choice = process_var.get()

    if not player_choice or not task_choice:
        messagebox.showwarning("Warning", "Please select both a player and a task.")
    else:
        # You playを選ぶ
        if player_choice == "You play":
            # taskを選ぶ
            Youplay(task_choice)
        
        # Agt playを選ぶ
        else:         
            Agtplay(player_choice, task_choice, process_choice)
            
            
if __name__ == '__main__':
    root = tk.Tk()
    root.title("Option Selector")

    player_var = tk.StringVar(value="")
    task_var = tk.StringVar(value="")
    process_var = tk.StringVar(value="")

    tk.Label(root, text="Please choose a player option:").pack()
    player_options = ["You play", "tableQ", "netQ", "replayQ", "targetQ"]
    for option in player_options:
        tk.Radiobutton(root, text=option, variable=player_var, value=option).pack(anchor=tk.W)

    tk.Label(root, text="Please choose a task option:").pack()
    task_options = ["no_wall", "fixed_wall", "random_wall"]
    for option in task_options:
        tk.Radiobutton(root, text=option, variable=task_var, value=option).pack(anchor=tk.W)
    
    tk.Label(root, text="Please choose a process option:").pack()
    process_options = ["learn", "more", "graph", "anime"]
    for option in process_options:
        tk.Radiobutton(root, text=option, variable=process_var, value=option).pack(anchor=tk.W)

    tk.Button(root, text="start", command=on_select).pack()

    root.mainloop()

C:\Users\msy-t\anaconda3\envs\DL\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


list index out of range
エージェントのパラメータがロードできません
outs
[(array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 2, -0.1, False, array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])), (array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0,

outs
[(array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 2, -0.1, False, array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])), (array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]]

outs
[(array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 2, -0.1, False, array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])), (array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]]

outs
[(array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 2, -0.1, False, array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])), (array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]]

...

outs
[(array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 0, -0.1, False, array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])), (array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]]

outs
[(array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 2, -0.1, False, array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])), (array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]]

outs
[(array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 2, -0.1, False, array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])), (array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]]

outs
[(array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 1, -0.2, True, array([[9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9]])), (array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

outs
[(array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 2, -0.1, False, array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])), (array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]]